Pistes :
- Trouver d'autres paramètres avec lesquels jouer

In [1]:
import sentencepiece as spm
import pandas as pd

# Spécifiez le chemin vers votre fichier CSV
chemin_fichier_csv = 'jorf_2023.csv'

# Spécifiez le séparateur (dans ce cas, '|')
separateur = '|'

# Utilisez Pandas pour lire le fichier CSV dans un DataFrame
df = pd.read_csv(chemin_fichier_csv, sep=separateur)

# Affichez le DataFrame
print(df)


        JORFTEXT000048734585            Unnamed: 1  0 Unnamed: 3  0.1  \
0       JORFTEXT000048734585  JORFVERS000048734585  0        NaN    0   
1       JORFTEXT000048734585  JORFARTI000048734586  1          1    1   
2       JORFTEXT000048734585  JORFARTI000048734586  1          1    2   
3       JORFTEXT000048734585  JORFARTI000048734586  1          1    3   
4       JORFTEXT000048734585  JORFARTI000048734586  1          1    4   
...                      ...                   ... ..        ...  ...   
454295  JORFTEXT000046851183  JORFVERS000046851183  0        NaN    0   
454296  JORFTEXT000046851183  JORFARTI000046851184  1        NaN    1   
454297  JORFTEXT000046851186                   NaN  0        NaN    0   
454298  JORFTEXT000046851186  JORFVERS000046851186  0        NaN    0   
454299  JORFTEXT000046851186  JORFARTI000046851187  1        NaN    1   

                           fr/lr/loi/2023-1380/2023-12-31  
0       LOI n° 2023-1380 du 30 décembre 2023 visant à ...  
1  

In [2]:
import re

def extraire_texte_entre_symboles(texte):
    # Utilisez une expression régulière pour extraire le texte entre '«' et '»'
    #match = re.search(r'«(.+)»', texte)
    match = re.search(r'«(.+)', texte) # Beaucoup de phrases n'ont pas le '»' nécessaire
    
    # Vérifiez si une correspondance a été trouvée
    if match:
        texte_extrait = match.group(1)
        return texte_extrait
    else:
        return None

def extraire_texte_en_dehors_des_symboles(texte):
    # Utilisez une expression régulière pour extraire le texte entre '«' et '»'
    match = re.search(r'^[^«]+', texte)
    
    # Vérifiez si une correspondance a été trouvée
    if match:
        texte_extrait = match.group(0)
        return texte_extrait
    else:
        return None

# Exemple d'utilisation
ma_chaine = "Ceci est un exemple «de texte «à extraire» avec des guillemets «pour tester»»."
resultat = extraire_texte_entre_symboles(ma_chaine)

if resultat:
    print("Texte extrait :", resultat)
else:
    print("Aucun texte trouvé entre les symboles '«' et '»'.")
    
resultat = extraire_texte_en_dehors_des_symboles(ma_chaine)

if resultat:
    print("Texte extrait :", resultat)
else:
    print("Aucun texte trouvé en dehors des symboles '«' et '»'.")



Texte extrait : de texte «à extraire» avec des guillemets «pour tester»».
Texte extrait : Ceci est un exemple 


In [3]:
# Création des colonnes ne contenant que les textes relatifs au Droit/non Droit
df['droit'] = df['fr/lr/loi/2023-1380/2023-12-31'].apply(extraire_texte_entre_symboles)
df['non_droit'] = df['fr/lr/loi/2023-1380/2023-12-31'].apply(extraire_texte_en_dehors_des_symboles)


In [5]:
# Ecrire l'ensemble du contenu dans un fichier texte
with open("input_data/jorf_2023.txt","w") as f:
    f.writelines("\n".join(df["fr/lr/loi/2023-1380/2023-12-31"].to_list()))
    
# On extrait que le Droit : entre guillemets
with open("./input_data/jorf_2023_droit.txt","w") as f:
    data = "\n".join(df["droit"].dropna().to_list())
    f.writelines(data)
    
# On extrait que ce qui entoure le Droit
with open("./input_data/jorf_2023_non_droit.txt","w") as f:
    data = "\n".join(df["non_droit"].dropna().to_list())
    f.writelines(data)

    A. Nombre de tokens

In [6]:
for n in (100,1000,10000):
    spm.SentencePieceTrainer.train(input='./input_data/jorf_2023.txt', model_prefix=f'./models/{n}_tokens', vocab_size=n)

In [7]:
# On encode une phrase selon les différents modèles : 100 tokens appara^t clairement insuffisant
for n in [100,1000,10000]:
    sp = spm.SentencePieceProcessor(model_file=f'./models/{n}_tokens.model')
    print(f"Avec {n} tokens :")
    print(sp.EncodeAsPieces("Décret du 29 décembre 2023 portant promotion dans l'ordre national de la Légion d'honneur"))

Avec 100 tokens :
['▁', 'D', 'é', 'c', 're', 't', '▁d', 'u', '▁', '2', '9', '▁d', 'é', 'c', 'e', 'm', 'b', 're', '▁', '2', '0', '2', '3', '▁', 'p', 'o', 'r', 't', 'a', 'n', 't', '▁', 'p', 'r', 'o', 'm', 'o', 't', 'i', 'o', 'n', '▁d', 'a', 'n', 's', '▁', 'l', "'", 'o', 'r', 'd', 're', '▁', 'n', 'a', 't', 'i', 'o', 'n', 'a', 'l', '▁de', '▁', 'l', 'a', '▁', 'L', 'é', 'g', 'i', 'o', 'n', '▁d', "'", 'h', 'o', 'n', 'n', 'e', 'u', 'r']
Avec 1000 tokens :
['▁Décret', '▁du', '▁2', '9', '▁décembre', '▁2023', '▁portant', '▁pro', 'mo', 'tion', '▁dans', '▁l', "'", 'ord', 're', '▁national', '▁de', '▁la', '▁L', 'é', 'g', 'ion', '▁d', "'", 'h', 'onne', 'ur']
Avec 10000 tokens :
['▁Décret', '▁du', '▁29', '▁décembre', '▁2023', '▁portant', '▁promotion', '▁dans', '▁l', "'", 'ordre', '▁national', '▁de', '▁la', '▁Légion', '▁d', "'", 'honneur']


100 tokens sont bien sur insuffisants pour intélligement traiter ce lexique. Avec 1000 tokens on a encore quelques mots qu'on s'attendrait à voir pris dans l'intégralité et pourtant découpés, comme 'Légion' et 'honneur'. Avec 10000 tokens, la tokénisation est presque trop ample, en prenant '29' comme un token en soit.

    B. Droit/non droit

In [8]:
spm.SentencePieceTrainer.train(input='./input_data/jorf_2023_droit.txt', model_prefix='./models/droit', vocab_size=1000)
spm.SentencePieceTrainer.train(input='./input_data/jorf_2023_non_droit.txt', model_prefix='./models/non_droit', vocab_size=1000)

In [9]:
sp1 = spm.SentencePieceProcessor(model_file=f'./models/droit.model')
sp2 = spm.SentencePieceProcessor(model_file=f'./models/non_droit.model')

# Comparaison de la tokenisation d'une phrase de droit et d'une non de Droit 
s1 = "« L'exonération est attribuée ou retirée dans les conditions prévues aux articles R. 5553-1 et R. 5553-2. »"
s2 = "Arrêté du 26 décembre 2023 relatif aux prix des prestations d'hébergement de certains établissements accueillant des personnes âgées"

for s in [s1,s2]:
    print(s)
    for sp in [sp1,sp2]:
        print(sp.EncodeAsPieces(s))
    print("\n")

« L'exonération est attribuée ou retirée dans les conditions prévues aux articles R. 5553-1 et R. 5553-2. »
['▁«', '▁L', "'", 'ex', 'on', 'é', 'r', 'ation', '▁est', '▁', 'att', 'ri', 'b', 'u', 'ée', '▁ou', '▁re', 'ti', 'ré', 'e', '▁dans', '▁les', '▁conditions', '▁prévues', '▁aux', '▁articles', '▁R', '.', '▁5', '5', '5', '3-1', '▁et', '▁R', '.', '▁5', '5', '5', '3', '-2', '.', '▁»']
['▁', '«', '▁L', "'", 'ex', 'on', 'é', 'r', 'ation', '▁est', '▁', 'at', 't', 'ri', 'b', 'u', 'ée', '▁ou', '▁re', 'ti', 'ré', 'e', '▁dans', '▁les', '▁conditions', '▁prévu', 'es', '▁aux', '▁articles', '▁R', '.', '▁5', '5', '5', '3', '-1', '▁et', '▁R', '.', '▁5', '5', '5', '3', '-', '2', '.', '▁', '»']


Arrêté du 26 décembre 2023 relatif aux prix des prestations d'hébergement de certains établissements accueillant des personnes âgées
['▁A', 'r', 'r', 'ê', 'té', '▁du', '▁2', '6', '▁décembre', '▁2023', '▁relatif', '▁aux', '▁p', 'ri', 'x', '▁des', '▁prestation', 's', '▁d', "'", 'h', 'é', 'b', 'er', 'g', 'ement', 

Les tokenisations Droit/Non Droit ont chacune adopté des tokens adaptés à leur vocabuaire, comme par exemple 'attribué'/'Arrêté'